In [ ]:
# 初始化
import django_setup

## tiptap / docx转化测试

In [ ]:
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_doc_chap3 - 1pages_无嵌套 .docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_chp1_short.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_doc_chap3 - 1pages_有嵌套.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/caseTest/Case6_中国区零食包框架供应商采购项目招采文件.docx"
doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/实战：招标文件-第1包：一级压榨花生油 - 副本.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_chp3.docx"
#doc_path_win = "C:/Users/huiwa/Downloads/文本分析测试/test_files/parts_chp3 p1.docx"

# 转换为WSL路径
doc_path_wsl  = doc_path_win.replace("C:", "/mnt/c").replace("\\", "/")
print(f"WSL路径: {doc_path_wsl}")


import os
print(f"文件是否存在: {os.path.exists(doc_path_wsl)}")

In [ ]:
from apps.projects.tiptap.docx import docx_to_html
html_content = docx_to_html(doc_path_wsl)
from pprint import pprint
pprint(html_content)

In [ ]:
from IPython.display import display, HTML
display(HTML(html_content))

In [ ]:
from apps.projects.tiptap.docx import docx_to_tiptap_json
json_content = docx_to_tiptap_json(doc_path_wsl)
from pprint import pprint
pprint(json_content)

## 集成测试

In [2]:
from apps.projects.models import Project, TaskStatus

In [ ]:
from apps.projects.models import Task, TaskType
task = Task.objects.get(stage__project=current_project, type=TaskType.DOCX_EXTRACTION_TASK)
print(task.docx_tiptap)

In [ ]:
current_project = Project.objects.get(project_name='2')
print(f"用于测试的项目: {current_project.project_name}")
print(f"项目包含的文件: {current_project.files.all()}")


In [ ]:
task.docx_tiptap = "1"
task.save()
print(task.docx_tiptap)

In [ ]:
from django.core.exceptions import ValidationError
import os, tempfile, requests, uuid
import logging
logger = logging.getLogger(__name__)

# 获取文件的预签名URL
presigned_url = current_project.files.first().get_presigned_url()
if not presigned_url:
    raise ValidationError("无法获取文件访问地址")

logger.info(f"开始下载文件: project_id={current_project.id}, file={current_project.files.first().name}")

# 下载文件到临时文件
temp_file_path = os.path.join(tempfile.gettempdir(), f"doc_analysis_{uuid.uuid4()}.docx")
response = requests.get(presigned_url)
response.raise_for_status()

with open(temp_file_path, 'wb') as temp_file:
    temp_file.write(response.content)

In [ ]:
task.docx_tiptap = json_content
task.save()
print(task.docx_tiptap)